#Food recommender system

In [ ]:
#import library
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
Menu = pd.read_csv("/content/drive/MyDrive/Hệ khuyến nghị/FoodUITDataset - Final.csv")#đặt link data
Menu.head()

,ID món ăn,Tên món ăn,Thể loại món ăn,Nguyên liệu chính,ID bữa ăn,Thể loại bữa ăn,Giá ước tính,Vùng miền,clone
0,1,Phở Cuốn Cùng Nước Chấm,Ăn Vặt,"Tỏi, ớt, chanh, bánh phở, cà rốt, ngò, dưa leo...",6,Mặn,"42,051",Bắc,Ăn Vặt
1,2,Bánh Đúc Mặn,Ăn Vặt,"100gr bột gạo, 180gr thịt heo xay, 200ml nước ...",1,Mặn,"93,710",Ba miền,Ăn Vặt
2,3,Bánh Đúc Ngọt,Ăn Vặt,"Bột năng 120 gr, bột gạo 80 gr, tinh bột bắp 1...",1,Chay,"147,320",Ba miền,Ăn Vặt
3,4,Nem Lụi Nướng Và Nước Chấm,Ăn Vặt,"600gr thịt lợn, 150gr tôm, 1 xấp bánh tráng, đ...",6,Mặn,"98,087",Trung,Ăn Vặt
4,5,Cá Viên Chiên Giòn,Ăn Vặt,"300gr cá phi lê, 3 muỗng bột bắp, 1 túi bột ch...",4,Mặn,"111,663",Ba miền,Ăn Vặt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Thông tin dữ liệu


In [ ]:
#Đếm số lượng các nhãn Thể loại món ăn
food_category_count = Menu['Thể loại món ăn'].value_counts()
food_category_count

Dân dã                   99
Xào                      96
Chiên                    92
Nướng                    79
Kho                      76
Lẩu                      71
Canh                     65
Dân Dã                   46
Gỏi                      43
Cháo                     41
Chè                      41
Ăn Vặt                   39
Xôi                      29
Giải khát-Tráng miệng    25
Hấp                      23
Súp                      22
Bún                      20
Hầm                      17
Trộn                     15
Rang                     12
Luộc                     11
Mứt                       9
Ăn vặt                    7
Chả                       2
Name: Thể loại món ăn, dtype: int64

In [ ]:
#Đếm số lượng các nhãn ID bữa ăn
meal_ID_count = Menu['ID bữa ăn'].value_counts()
meal_ID_count

6      311
7      298
14     220
5       58
8       51
15      20
1        9
4        8
12       3
13       1
141      1
Name: ID bữa ăn, dtype: int64

In [ ]:
#Đếm số món riêng biệt
n_dish_uniq = Menu['Tên món ăn'].nunique()
n_dish_uniq

969

In [ ]:
#Đếm số lượng các nhãn Thể loại món ăn
food_category_count = Menu['Thể loại bữa ăn'].value_counts()
food_category_count

Mặn            806
Chay           108
Chay và mặn     41
Name: Thể loại bữa ăn, dtype: int64

#System



##Quan sát dữ liệu

In [ ]:
#xóa clone
Menu = Menu.drop(columns=['clone'])
Menu

,ID món ăn,Tên món ăn,Thể loại món ăn,Nguyên liệu chính,ID bữa ăn,Thể loại bữa ăn,Giá ước tính,Vùng miền
0,1,Phở Cuốn Cùng Nước Chấm,Ăn Vặt,"Tỏi, ớt, chanh, bánh phở, cà rốt, ngò, dưa leo...",6,Mặn,"42,051",Bắc
1,2,Bánh Đúc Mặn,Ăn Vặt,"100gr bột gạo, 180gr thịt heo xay, 200ml nước ...",1,Mặn,"93,710",Ba miền
2,3,Bánh Đúc Ngọt,Ăn Vặt,"Bột năng 120 gr, bột gạo 80 gr, tinh bột bắp 1...",1,Chay,"147,320",Ba miền
3,4,Nem Lụi Nướng Và Nước Chấm,Ăn Vặt,"600gr thịt lợn, 150gr tôm, 1 xấp bánh tráng, đ...",6,Mặn,"98,087",Trung
4,5,Cá Viên Chiên Giòn,Ăn Vặt,"300gr cá phi lê, 3 muỗng bột bắp, 1 túi bột ch...",4,Mặn,"111,663",Ba miền
...,...,...,...,...,...,...,...,...
975,976,Trà tắc,Giải khát-Tráng miệng,NaN,7,NaN,"58,784",Ba miền
976,977,Yaourt,Giải khát-Tráng miệng,NaN,7,NaN,"56,141",Ba miền
977,978,Hồng trà,Giải khát-Tráng miệng,NaN,7,NaN,"33,655",Ba miền
978,979,Chè đậu xanh,Giải khát-Tráng miệng,NaN,7,NaN,"66,382",Ba miền


In [ ]:
Menu['Thể loại món ăn'].value_counts()

Dân dã                   99
Xào                      96
Chiên                    92
Nướng                    79
Kho                      76
Lẩu                      71
Canh                     65
Dân Dã                   46
Gỏi                      43
Cháo                     41
Chè                      41
Ăn Vặt                   39
Xôi                      29
Giải khát-Tráng miệng    25
Hấp                      23
Súp                      22
Bún                      20
Hầm                      17
Trộn                     15
Rang                     12
Luộc                     11
Mứt                       9
Ăn vặt                    7
Chả                       2
Name: Thể loại món ăn, dtype: int64

In [ ]:
Menu['ID bữa ăn'].value_counts()

6      311
7      298
14     220
5       58
8       51
15      20
1        9
4        8
12       3
13       1
141      1
Name: ID bữa ăn, dtype: int64

In [ ]:
Menu['Thể loại bữa ăn'].value_counts()

Mặn            806
Chay           108
Chay và mặn     41
Name: Thể loại bữa ăn, dtype: int64

##Rating

In [ ]:
#Tính độ tương quan để tạo thành 1 bàn ăn 
Menu_dessert = Menu[Menu['Thể loại món ăn'] == "Giải khát-Tráng miệng"]
Menu_dessert

,ID món ăn,Tên món ăn,Thể loại món ăn,Nguyên liệu chính,ID bữa ăn,Thể loại bữa ăn,Giá ước tính,Vùng miền
955,956,Rau câu dừa,Giải khát-Tráng miệng,NaN,15,NaN,"53,933",Ba miền
956,957,Chè hạt sen,Giải khát-Tráng miệng,NaN,15,NaN,"129,770",Ba miền
957,958,Cocacola,Giải khát-Tráng miệng,NaN,15,NaN,"130,137",Ba miền
958,959,Pepsi,Giải khát-Tráng miệng,NaN,15,NaN,"24,711",Ba miền
959,960,Sprite,Giải khát-Tráng miệng,NaN,15,NaN,"53,903",Ba miền
960,961,7-UP,Giải khát-Tráng miệng,NaN,15,NaN,"51,544",Ba miền
961,962,Sting dâu,Giải khát-Tráng miệng,NaN,15,NaN,"133,346",Ba miền
962,963,Rau câu khoai môn,Giải khát-Tráng miệng,NaN,15,NaN,"78,653",Ba miền
963,964,Nước tăng lực Warrior,Giải khát-Tráng miệng,NaN,15,NaN,"52,785",Ba miền
964,965,Nước bí đao Wonderfarm,Giải khát-Tráng miệng,NaN,15,NaN,"62,853",Ba miền


In [ ]:
Menu_Broth = Menu[Menu['Thể loại món ăn'] == "Canh"]
Menu_Pot = Menu[Menu['Thể loại món ăn'] == "Lẩu"]
Menu_Broth_Pot = pd.concat([Menu_Broth,Menu_Pot])
Menu_Broth_Pot

,ID món ăn,Tên món ăn,Thể loại món ăn,Nguyên liệu chính,ID bữa ăn,Thể loại bữa ăn,Giá ước tính,Vùng miền
58,59,Canh Kim Chi Đậu Phụ Và Thịt Bò,Canh,"Thịt bò 300gr, kim chi 300gr, 2 miếng đậu phụ,...",6,Mặn,"98,643",Hàn
59,60,Canh Ghẹ Rau Muống,Canh,"3-4 con Ghẹ, 1 mớ rau muống, hành tím, tỏi, ớt",6,Mặn,"141,405",Ba miền
60,61,Canh Ghẹ Măng Chua,Canh,"Ghẹ hoặc cua to, 1/2 quả dứa, 2 quả cà chua, m...",6,Mặn,"25,600",Ba miền
61,62,Canh Chua Cá Điêu Hồng,Canh,"Cá diêu hồng 1 con, cà chua 2 quả, thơm 1/4 t...",6,Mặn,"23,044",Nam
62,63,"Canh Bí Đỏ Hầm Xương, Thịt Bằm, Tôm Khô",Canh,"100gr Tôm khô, 100gr thịt heo xay, 600gr bí đỏ...",6,Mặn,"36,370",Ba miền
...,...,...,...,...,...,...,...,...
788,789,Lẩu bò nấm,Lẩu,"Bò, cải cúc, hành tây, rong biển, ớt, mắm tép",14,Mặn,"146,296",Ba miền
789,790,Lẩu gà,Lẩu,"Gà, hành, sả, chanh, tỏi, ớt, màu điều",14,Mặn,"87,566",Ba miền
790,791,Lẩu Thái thập cẩm,Lẩu,"xương ống, cà chua, sả, cốt chanh, riềng, dứa,...",14,Mặn,"119,604",Ba miền
791,792,Lẩu gà lá é,Lẩu,"Gà, lá é, măng chua, chanh, tỏi, ớt hiểm",14,Mặn,"145,487",Ba miền


In [ ]:
Menu_main_dish = Menu.copy()
index_2_dish = Menu_main_dish[(Menu_main_dish['Thể loại món ăn'] == 'Canh')|(Menu_main_dish['Thể loại món ăn'] == 'Lẩu')|(Menu_main_dish['Thể loại món ăn'] == 'Giải khát-Tráng miệng')].index
Menu_main_dish.drop(index_2_dish , inplace=True)

In [ ]:
Menu_main_dish

,ID món ăn,Tên món ăn,Thể loại món ăn,Nguyên liệu chính,ID bữa ăn,Thể loại bữa ăn,Giá ước tính,Vùng miền
0,1,Phở Cuốn Cùng Nước Chấm,Ăn Vặt,"Tỏi, ớt, chanh, bánh phở, cà rốt, ngò, dưa leo...",6,Mặn,"42,051",Bắc
1,2,Bánh Đúc Mặn,Ăn Vặt,"100gr bột gạo, 180gr thịt heo xay, 200ml nước ...",1,Mặn,"93,710",Ba miền
2,3,Bánh Đúc Ngọt,Ăn Vặt,"Bột năng 120 gr, bột gạo 80 gr, tinh bột bắp 1...",1,Chay,"147,320",Ba miền
3,4,Nem Lụi Nướng Và Nước Chấm,Ăn Vặt,"600gr thịt lợn, 150gr tôm, 1 xấp bánh tráng, đ...",6,Mặn,"98,087",Trung
4,5,Cá Viên Chiên Giòn,Ăn Vặt,"300gr cá phi lê, 3 muỗng bột bắp, 1 túi bột ch...",4,Mặn,"111,663",Ba miền
...,...,...,...,...,...,...,...,...
950,951,Mướp đắng xào trứng,Xào,"Mướp đắng, trứng",6,Chay,"108,485",Ba miền
951,952,Xôi đậu xanh,Xôi,"Gạo nếp, đậu xanh, muối, dầu ăn",1,Chay,"79,568",Ba miền
952,953,Xôi sắn mỡ hành,Xôi,"Củ sắn, thịt mỡ, hành lá, lạp xưởng",1,Mặn,"80,046",Ba miền
953,954,Xôi ngô,Xôi,"Gạo nếp, ngô ngọt, nước cốt dừa, dừa sợi",1,Chay,"111,400",Ba miền


In [ ]:
Menu_main_dish['Thể loại món ăn'].value_counts()

Dân dã    99
Xào       96
Chiên     92
Nướng     79
Kho       76
Dân Dã    46
Gỏi       43
Cháo      41
Chè       41
Ăn Vặt    39
Xôi       29
Hấp       23
Súp       22
Bún       20
Hầm       17
Trộn      15
Rang      12
Luộc      11
Mứt        9
Ăn vặt     7
Chả        2
Name: Thể loại món ăn, dtype: int64

##Searching a single dishes

In [ ]:
#Hàm xử lý tìm kiếm một món ăn bất kỳ trong Menu
def SearchProcessing(dishes):
  pro_dishes = dishes
  count = 0
  while (count < 1):#Count = 0 thì có thể chạy lại
    #print("point...")
    for x in Menu["Tên món ăn"]:
      if x == pro_dishes:
        print(Menu[["ID món ăn","Tên món ăn"]].loc[Menu["Tên món ăn"] == x].to_markdown())
        count = count + 1 #Để cho count khác 0 để kết thúc vòng lặp while nhưng vẫn sẽ chạy hết for
    if(count == 0):
      print(" - Không có món ăn trên trên cơ sở dữ liệu!!!")
      time.sleep(0.5)
      count = int(input("Nhập ""1"" để thoát và ""0"" để tiếp tục: "))#Tùy chọn để thoát vòng lặp nếu không muốn nhập nữa...
      if count == 0:
       pro_dishes = input("Nhập tên món ăn:")
  return Menu.loc[Menu["Tên món ăn"] == pro_dishes]

In [ ]:
def DishTransfer(dishes):
  pro_dishes = dishes
  count = 0
  while (count < 1):#Count = 0 thì có thể chạy lại
    for x in Menu["Tên món ăn"]:
      if x == pro_dishes:
        count = count + 1 #Để cho count khác 0 để kết thúc vòng lặp while nhưng vẫn sẽ chạy hết for
    if(count == 0):
      print(" - Không có món ăn trên trên cơ sở dữ liệu!!!")
      time.sleep(0.5)
      count = int(input("Nhập ""1"" để thoát và ""0"" để tiếp tục: "))#Tùy chọn để thoát vòng lặp nếu không muốn nhập nữa...
      if count == 0:
       pro_dishes = input("Nhập tên món ăn:")
  return Menu.loc[Menu["Tên món ăn"] == pro_dishes]

In [ ]:
#viết hàm tìm kiếm món ăn trong cơ sở dữ liệu
def TraketQua():
  dishes = input("Nhập tên món ăn:")
  print("Đang tìm kiếm trong cơ sở dữ liệu...")
  time.sleep(1)
  print("-----------------------------------------")
  print("Kết quả:")
  Test = SearchProcessing(dishes)
  print("Đã tìm kiếm xong!!!")
  return dishes

In [ ]:
TraketQua()

Nhập tên món ăn:Trà đào
Đang tìm kiếm trong cơ sở dữ liệu...
-----------------------------------------
Kết quả:
|     |   ID món ăn | Tên món ăn   |
|----:|------------:|:-------------|
| 974 |         975 | Trà đào      |
Đã tìm kiếm xong!!!


'Trà đào'

##Reccomender Sys

In [ ]:
#Bộ đếm
def verify_counter():
  verify_df = pd.DataFrame(index = ['broth_pot_counter','dessert_counter','main_dishes'])
  counter = [1,1,2]
  verify_df['counter'] = counter
  return verify_df

In [ ]:
#Xử lý phân loại để lấy kết quả đầu ra nạp cho bộ đếm
def ID_checking(dishes):
  print(dishes)
  if dishes.iloc[0]['Thể loại món ăn'] == 'Lẩu' or dishes.iloc[0]['Thể loại món ăn'] == 'Canh':
    result = "broth_pot"
    return result 
  elif dishes.iloc[0]['Thể loại món ăn'] == 'Giải khát-Tráng miệng':
    result = "dessert"
    return result 
  else:
    result = "main"
    return result

In [ ]:
#Giảm bộ đếm
def counter_reduction(result,verify_df):
  if(result == "broth_pot"):
    verify_df['counter'].iloc[0] -= 1
  if(result == "dessert"):
    verify_df['counter'].iloc[1]-=1
  if(result == "main"):
    verify_df['counter'].iloc[2]-=1  
  if verify_df['counter'].iloc[0]  == 0 & verify_df['counter'].iloc[1] == 0 & verify_df['counter'].iloc[2] == 0 :
    print("Đã hoàn thành yêu cầu...")

In [ ]:
#phase 1
def phase1(first_dish):
  if first_dish.iloc[0]['Thể loại món ăn'] == 'Giải khát-Tráng miệng':
    no_dessert_broth_pot = Menu_Broth_Pot.sample(n = 5)
    no_dessert_main_dish = Menu_main_dish.sample(n = 10)
    global no_dessert 
    no_dessert = pd.concat([no_dessert_broth_pot, no_dessert_main_dish])
    print(no_dessert[["ID món ăn","Tên món ăn"]].to_markdown())
    nodep1 = int(input('Hãy chọn ID món ăn tiếp theo'))
    first_dish.loc[no_dessert.index[nodep1]] = no_dessert.iloc[nodep1]
    print(first_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    return first_dish
  elif (first_dish.iloc[0]['Thể loại món ăn'] == 'Lẩu' or first_dish.iloc[0]['Thể loại món ăn'] == 'Canh'):
    no_broth_pot_dessert = Menu_dessert.sample(n = 5)
    no_broth_pot_main_dish = Menu_main_dish.sample(n = 10)
    global no_broth_pot
    no_broth_pot = pd.concat([no_broth_pot_dessert, no_broth_pot_main_dish])
    print(no_broth_pot[["ID món ăn","Tên món ăn"]].to_markdown())
    nodep1 = int(input('Hãy chọn ID món ăn tiếp theo'))
    first_dish.loc[no_broth_pot.index[nodep1]] = no_broth_pot.iloc[nodep1]
    print(first_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    return first_dish
  else:
    no_main_dish_dessert = Menu_dessert.sample(n = 7)
    no_main_dish_broth_pot = Menu_Broth_Pot.sample(n = 8)
    global no_main_dish
    no_main_dish = pd.concat([no_main_dish_dessert,no_main_dish_broth_pot])
    print(no_main_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    nodep1 = int(input('Hãy chọn ID món ăn tiếp theo'))
    first_dish.loc[no_main_dish.index[nodep1]] = no_main_dish.iloc[nodep1]
    print(first_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    return first_dish

In [ ]:
#Phase 2
def phase2( counter):
  if (counter['counter'].iloc[0] == 0 & counter['counter'].iloc[1] == 0):
    Menu_main_dish = Menu_main_dish.sample(n = 15)
    print(Menu_main_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    nodep2 = int(input('Hãy chọn ID món ăn tiếp theo'))
    second_dish = Menu_main_dish.iloc[nodep2]
    #second_dish = Menu_main_dish.loc[Menu_main_dish.index[nodep2]] = Menu_main_dish.iloc[nodep2]
    second_dish = second_dish.loc[Menu_main_dish.index[nodep2]]
    print(second_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    return second_dish
  if(counter['counter'].iloc[2] == 0):
    print(no_main_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    nodep2 = int(input('Hãy chọn ID món ăn tiếp theo'))
    second_dish= no_main_dish.iloc[nodep2]
    #second_dish.loc[no_main_dish.index[nodep2]] = no_main_dish.iloc[nodep2]
    second_dish=second_dish.loc[no_main_dish.index[nodep2]]
    print(second_dish[["ID tmón ăn","Tên món ăn"]].to_markdown())
    return second_dish
  if(counter['counter'].iloc[0] == 0 & counter['counter'].iloc[2] == 1):
    print(no_dessert[["ID món ăn","Tên món ăn"]].to_markdown())
    nodep2 = int(input('Hãy chọn ID món ăn tiếp theo'))
    second_dish = no_dessert.iloc[nodep2]
    #second_dish.loc[no_dessert.index[nodep2]] = no_dessert.iloc[nodep2]
    second_dish = second_dish.loc[no_dessert.index[nodep2]]
    print(second_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    return second_dish
  if (counter['counter'].iloc[1] == 0 & counter['counter'].iloc[2] == 1):
    print(no_broth_pot[["ID món ăn","Tên món ăn"]].to_markdown())
    nodep2 = int(input('Hãy chọn ID món ăn tiếp theo'))
    second_dish = no_broth_pot.iloc[nodep2]
    #second_dish = no_broth_pot.loc[no_broth_pot.index[nodep2]] = no_broth_pot.iloc[nodep2]
    second_dish = second_dish.loc[no_broth_pot.index[nodep2]]
    print(second_dish[["ID món ăn","Tên món ăn"]].to_markdown())
    return second_dish

##main

In [ ]:
def OutputDish():
  counter = verify_counter()
  static_menu = pd.DataFrame(columns=['ID món ăn', 'Tên món ăn', 'Thể loại món ăn', 'Nguyên liệu chính', 'ID bữa ăn', 'Thể loại bữa ăn', 'Giá ước tính', 'Vùng miền'])
  #1st
  first_dish = TraketQua()
  first_dish_df = DishTransfer(first_dish)
  #No1_dishes = ID_checking(first_dish_df)
  #counter_reduction(No1_dishes, counter)
  Static_menu = pd.concat([static_menu, first_dish_df])
  #2nd
  Second_dishes_df = phase1(first_dish_df)
  #No2_dishes = ID_checking(Second_dishes_df)
  #counter_reduction(No2_dishes, counter)
  Static_menu = pd.concat([static_menu, Second_dishes_df])
  #3rd
  third_dishes_df = phase2(counter)
  #No3_dishes = ID_checking(third_dishes_df)
  #counter_reduction(No3_dishes, counter)
  Static_menu = pd.concat([static_menu, third_dishes_df])
 

In [ ]:
OutputDish()

Nhập tên món ăn:Trà đào
Đang tìm kiếm trong cơ sở dữ liệu...
-----------------------------------------
Kết quả:
|     |   ID món ăn | Tên món ăn   |
|----:|------------:|:-------------|
| 974 |         975 | Trà đào      |
Đã tìm kiếm xong!!!
|     |   ID món ăn | Tên món ăn                    |
|----:|------------:|:------------------------------|
| 457 |         458 | Canh bí đỏ nấu tôm            |
| 284 |         285 | Lẩu gà lá giang               |
| 292 |         293 | lẩu gà bỗng rượu              |
|  67 |          68 | Canh Rong Biển Khô            |
| 308 |         309 | lẩu cá thác lác khổ qua       |
| 714 |         715 | Cút rôti nước dừa             |
| 795 |         796 | Gan Lợn Luộc                  |
| 902 |         903 | Măng tây xào thịt bò          |
| 592 |         593 | Trứng vịt lộn                 |
| 646 |         647 | Dạ dày lợn luộc               |
| 529 |         530 | Cá Trắm Kho                   |
| 702 |         703 | Gà hầm thuốc bắc              |
|

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
